# Problem 6

We want to implement P1-FEM for a general advection-diffusion-reaction problem in 2D:
$$
    \begin{cases}
        -\textup{div}\left(\textup{A}\nabla u\right)+\beta\cdot\left(\nabla u\right)+\gamma u=f\quad&\text{in }\Omega=(0,1)\times(0,1),\\
        u=0,\quad&\text{on }\Gamma_D=(\{0\}\times(0,1))\cup((0,1)\times\{0,1\}),\\
        \left(\textup{A}\nabla u\right)\cdot\nu=g,\quad&\text{on }\Gamma_N=\{1\}\times(0,1),\\
    \end{cases}
$$
with
$$
    f(x,y)=100\exp\left\{-50\left((x-0.2)^2+(y-0.3)^2\right)\right\},\ g(x,y)=0,
$$
$$
    \textup{A}=\begin{bmatrix}
            0.2&-0.01\\-0.01&0.2
    \end{bmatrix},\ \beta=\begin{bmatrix}
            3\\0.3
    \end{bmatrix},\ \text{and}\ \gamma=0.05.
$$

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

## (a)

Derive a weak (Galerkin) formulation for the problem.

* $V=$...
* $a(u,v)=$...
* $b(v)=$...

The problem is well-posed because...

## (b)

In [ ]:
# Modify createmesh2D to make the top boundary Dirichlet.
def createmesh2D(h, Omega=(0, 1)):
    """
    This function (copied from ex 5) creates a mesh, flagging the bottom and left
    boundaries as Dirichlet and the top and right boundaries as Neumann.
    !!!!Modify this function to make the top boundary Dirichlet!!!!
    """
    L = Omega[1] - Omega[0] # square side length
    N = int(np.ceil(L / h)) # number of cells per side

    grid_1d = np.linspace(*Omega, N + 1)
    grid = np.stack((np.tile(grid_1d, N + 1),  # mesh vertices, sorted from left to right, from bottom to top
                     np.repeat(grid_1d, N + 1)), axis = 1)

    elements = [[(N + 1) * j + i, # all elements whose right angle corner is on the bottom left
                 (N + 1) * j + i + 1,
                 (N + 1) * (j + 1) + i] for j in range(N) for i in range(N)]
    elements += [[(N + 1) * (j + 1) + i + 1, # all elements whose right angle corner is on the top right
                  (N + 1) * (j + 1) + i,
                  (N + 1) * j + i + 1] for j in range(N) for i in range(N)]

    edges = [[(N + 1) * (N - j), (N + 1) * (N - j - 1)] for j in range(N)] # left side
    edges += [[j, j + 1] for j in range(N)] # bottom side
    edges += [[(N + 1) * (j + 1) - 1, (N + 1) * (j + 2) - 1] for j in range(N)] # right side
    edges += [[(N + 1) * (N + 1) - 1 - j, (N + 1) * (N + 1) - 2 - j] for j in range(N)] # top side

    boundaries = [1] * (2 * N) + [2] * (2 * N)
    return grid, elements, edges, boundaries

grid, elements, edges, boundaries = createmesh2D(.25)
print(f"{grid = }\n\n{elements = }\n\n{edges = }\n\n{boundaries = }")

plt.figure()
for j, pt in enumerate(grid):
    plt.text(* pt, f"DoF {j}")
for j, el in enumerate(elements):
    plt.plot([grid[i, 0] for i in el + [el[0]]],
             [grid[i, 1] for i in el + [el[0]]], 'c')
    plt.text((2 * grid[el[0], 0] + grid[el[1], 0]) / 3, 
             (2 * grid[el[0], 1] + grid[el[2], 1]) / 3,
             f"elem {j}", ha="center", va="center")
for ed, d in zip(edges, boundaries):
    color = "k" if d == 1 else "b"
    plt.plot([grid[i, 0] for i in ed],
             [grid[i, 1] for i in ed], color + "-")
plt.plot(*grid.T, 'go')
plt.xlim(-.01, 1.01), plt.ylim(-.01, 1.01)
plt.xlabel("x"), plt.ylabel("y")
plt.show()

In [ ]:
def assemble_LHS_poisson(mesh, A, beta, gamma):
    """
    Function to assemble P1-FEM LHS.
    """
    grid, elements, edges, boundaries = mesh
    ...
    mat = np.zeros((len(grid), len(grid)))
    for el in elements: # volume terms (must integrate in 2D)
        ...
    for ed, b in zip(edges, boundaries): # Dirichlet BCs (easy) or Robin BCs (must integrate in 1D)
        # ed is an edge (i.e., a pair of DoF indices), b is boundary flag
        ...
    return mat

In [ ]:
# check your implementation of assemble_LHS_poisson
from pickle import dump, load
mat = assemble_LHS_poisson(createmesh2D(0.5), np.array([[5, -1], [-1, 4]]),
                           np.array([-1, 2]), 0.3)
with open("../check/assemble_LHS_poisson_2D_advection.pkl", "rb") as f: mat_ex = load(f)
print(f"stiffness matrix should be {mat_ex}\ncomputed stiffness matrix is {mat}")

In [ ]:
# quadrature1D is needed (also by assemble_RHS_poisson) to implement natural (Neumann&Robin) BCs
def quadrature1D(domain, integrand):
    """
    Function to perform quadrature of callable function integrand over interval specified by domain.
    Note: here domain is a straight line segment in 2D.
    For instance, domain = [(x0, y0), (x1, y1)], the line segment's endpoints.
    Then a possible parametrization of the line segment is
        t -> (x(t), y(t)) := (x0, y0) + t * (x1 - x0, y1 - y0)
        t \in [0,1]
    We want to approximate
        \int_0^1 integrand(x(t), y(t)) * JAC * dt
    with JAC = ||(x1 - x0, y1 - y0)||.
    You can copy it over from ex 5.
    """
    ...

def quadrature2D(domain, integrand):
    """
    Function to perform quadrature of callable function integrand over simplices specified by domain.
    Note: here domain is a straight line segment in 2D.
    For instance, domain = [(x0, y0), (x1, y1), (x2, y2)], simplex's vertices.
    Then a possible parametrization of the simplex is
        (s, t) -> (x(s, t), y(s, t)) := (x0, y0) + s * (x1 - x0, y1 - y0) + t * (x2 - x0, y2 - y0),
        (s, t) such that 0 <= s <= 1 and 0 <= t <= 1 - s
    We want to approximate
        \int_0^1\int_0^{1-s} integrand(x(s, t), y(s, t)) * det(JAC) * dt * ds
    with det(JAC) = ||(x1 - x0, y1 - y0)||*||(x2 - x0, y2 - y0)||.
    You can copy it over from ex 5.
    """
    ...

def assemble_RHS_poisson(mesh, u0, f, g):
    """
    Function to assemble P1-FEM RHS.
    You can copy it over from ex 5.
    """
    grid, elements, edges, boundaries = mesh
    ...
    vec = np.zeros(len(grid))
    for el in elements: # volume terms (must integrate in 2D)
        ...
    for ed, b in zip(edges, boundaries): # Dirichlet BCs (easy) or natural BCs (must integrate in 1D)
        # ed is an edge (i.e., a pair of DoF indices), b is boundary flag
        ...
    return vec

Compute and plot the P1-FE solution.

In [ ]:
def solve_poisson(h):
    """
    Function to compute the P1-FEM solution uh with mesh-size h.
    """
    ...

def find_parent_element(h, x, Omega=(0, 1)):
    """
    Find the index of the element to which x belongs. x can be a single 2D point, or an array of points, with shape (M by 2).
    h is the mesh size used to generate the mesh over Omega, using the provided function.
    """
    x = np.asarray(x) # ensure x has the right format and shape
    if x.ndim == 1: x = x[np.newaxis, :]
    
    assert np.all(np.logical_and(*[np.logical_and(x[:, j] >= Omega[0],
                                                  x[:, j] <= Omega[1])
                                   for j in range(2)])), "some points are outside the domain"

    L = Omega[1] - Omega[0] # square side length
    N = int(np.ceil(L / h)) # number of cells per side

    vertices_1d = np.linspace(*Omega, N + 1)
    h_eff = vertices_1d[1] - vertices_1d[0] # actual grid size (after rounding of N)

    # find cartesian indices of x, i.e., indices of its square cell
    find_x = sum([x[:, 0] > v for v in vertices_1d[1 :]])
    find_y = sum([x[:, 1] > v for v in vertices_1d[1 :]])
    idx_elements = N * find_y + find_x

    # now check if x belongs to bottom-left- or top-right-pointing triangles
    x_flip = x[:, 0] + x[:, 1] > h_eff + vertices_1d[find_x] + vertices_1d[find_y]
    idx_elements[x_flip] += N ** 2
    return idx_elements

def eval_uh(mesh, uh, x):
    """
    Function to evaluate the P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    grid, elements, edges, boundaries = mesh
    h = ...
    Omega = ...
    idx_elements = find_parent_element(h, Omega, x)
    ...

def eval_uh_prime(mesh, uh, x):
    """
    Function to evaluate the first derivative of P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    grid, elements, edges, boundaries = mesh
    h = ...
    Omega = ...
    idx_elements = find_parent_element(h, Omega, x)
    ...
...

## (c)

Solve the problem with stronger advection terms.

In [ ]:
def solve_poisson(h):
    """
    Function to compute the P1-FEM solution uh with mesh-size h and increased flow speed beta.
    """
    ...
...